# Notebook to explore updating PNAS data
- rewrite code to make json not csv <br>
- what are we dropping city year combos in pdays routine?

In [1]:
#### Dependencies
import pandas as pd
import numpy as np
import xarray as xr
from random import random
from itertools import groupby
from operator import itemgetter
import geopandas as gpd 
from glob import glob
from statistics import mean
import julian
import time 
import multiprocessing as mp 
from multiprocessing import Pool
import os

In [2]:
# Data path
DATA_PATH = '/home/cascade/projects/UrbanHeat/data/'


## Let's write a script to make meta data

In [15]:
fn_in = DATA_PATH+'interim/GHS-UCDB-IDS.csv'

In [16]:
df = pd.read_csv(fn_in)

In [17]:
df.head()

,ID_HDC_G0,CTR_MN_NM,UC_NM_MN,GCPNT_LAT,GCPNT_LON
0,5782,Russia,Norilsk [RUS],69.333682,88.205172
1,3316,Russia,Murmansk [RUS],68.955354,33.078645
2,5645,Russia,Novy Urengoy [RUS],66.083799,76.646580
3,3185,Finland,Oulu [FIN],65.019378,25.482396
4,3539,Russia,Severodvinsk [RUS],64.572142,39.831477


In [8]:
ghs_fn = DATA_PATH+'raw/GHS_UCDB/GHS_STAT_UCDB2015MT_GLOBE_R2019A_V1_0.shp'

In [9]:
ghs = gpd.read_file(ghs_fn)

In [10]:
for col in ghs.columns:
    print(col)

ID_HDC_G0
QA2_1V
AREA
BBX_LATMN
BBX_LONMN
BBX_LATMX
BBX_LONMX
GCPNT_LAT
GCPNT_LON
CTR_MN_NM
CTR_MN_ISO
XBRDR
XCTR_NBR
XC_NM_LST
XC_ISO_LST
GRGN_L1
GRGN_L2
UC_NM_MN
UC_NM_LST
UC_NM_SRC
H75_NBR
H90_NBR
H00_NBR
H75_AREA
H90_AREA
H00_AREA
E_BM_NM_LS
E_SL_LST
EL_AV_ALS
E_KG_NM_LS
E_RB_NM_LS
E_WR_P_90
E_WR_P_00
E_WR_P_14
E_WR_T_90
E_WR_T_00
E_WR_T_14
B75
B90
B00
B15
P75
P90
P00
P15
BUCAP75
BUCAP90
BUCAP00
BUCAP15
NTL_AV
GDP90_SM
GDP00_SM
GDP15_SM
INCM_CMI
DEV_CMI
TT2CC
E_GR_AV90
E_GR_AV00
E_GR_AV14
E_GR_AH90
E_GR_AM90
E_GR_AL90
E_GR_AT90
E_GR_AH00
E_GR_AM00
E_GR_AL00
E_GR_AT00
E_GR_AH14
E_GR_AM14
E_GR_AL14
E_GR_AT14
E_EC2E_E75
E_EC2E_E90
E_EC2E_E00
E_EC2E_E12
E_EC2E_R75
E_EC2E_R90
E_EC2E_R00
E_EC2E_R12
E_EC2E_I75
E_EC2E_I90
E_EC2E_I00
E_EC2E_I12
E_EC2E_T75
E_EC2E_T90
E_EC2E_T00
E_EC2E_T12
E_EC2E_A75
E_EC2E_A90
E_EC2E_A00
E_EC2E_A12
E_EC2O_E75
E_EC2O_E90
E_EC2O_E00
E_EC2O_E12
E_EC2O_R75
E_EC2O_R90
E_EC2O_R00
E_EC2O_R12
E_EC2O_I75
E_EC2O_I90
E_EC2O_I00
E_EC2O_I12
E_EC2O_T75
E_EC2O_T90
E_EC2O_T

In [12]:
# columns we want 
cols = ['ID_HDC_G0','CTR_MN_NM', 'UC_NM_MN','GCPNT_LAT','GCPNT_LON']
out = ghs[cols]

In [14]:
# write out 
fn_out = DATA_PATH+'interim/GHS-UCDB-IDS.csv'
out.to_csv(fn_out, index = False)

In [19]:
for ctn in np.unique(out.CTR_MN_NM):
    print(ctn)

Afghanistan
Albania
Algeria
Angola
Argentina
Armenia
Australia
Austria
Azerbaijan
Bahamas
Bahrain
Bangladesh
Barbados
Belarus
Belgium
Belize
Benin
Bhutan
Bolivia
Bosnia and Herzegovina
Botswana
Brazil
Brunei
Bulgaria
Burkina Faso
Burundi
Cambodia
Cameroon
Canada
Cape Verde
Central African Republic
Chad
Chile
China
Colombia
Comoros
Costa Rica
Croatia
Cuba
Curaçao
Cyprus
Czech Republic
Côte d'Ivoire
Democratic Republic of the Congo
Denmark
Djibouti
Dominican Republic
Ecuador
Egypt
El Salvador
Equatorial Guinea
Eritrea
Estonia
Ethiopia
Fiji
Finland
France
French Polynesia
Gabon
Gambia
Georgia
Germany
Ghana
Greece
Guatemala
Guinea
Guinea-Bissau
Guyana
Haiti
Honduras
Hungary
Iceland
India
Indonesia
Iran
Iraq
Ireland
Israel
Italy
Jamaica
Japan
Jersey
Jordan
Kazakhstan
Kenya
Kosovo
Kuwait
Kyrgyzstan
Laos
Latvia
Lebanon
Lesotho
Liberia
Libya
Lithuania
Luxembourg
Macedonia
Madagascar
Malawi
Malaysia
Maldives
Mali
Malta
Mauritania
Mauritius
Mexico
Moldova
Mongolia
Montenegro
Morocco
Mozambique
M

In [20]:
regions_fn = DATA_PATH+'raw/countrylist.csv'
regions = pd.read_csv(regions_fn)

In [21]:
regions.head()

,name,alpha-2,alpha-3,country-code,iso_3166-2,region,sub-region,intermediate-region,region-code,sub-region-code,intermediate-region-code
0,Afghanistan,AF,AFG,4.0,ISO 3166-2:AF,Asia,Southern Asia,Southern Asia,142.0,34.0,NaN
1,Åland Islands,AX,ALA,248.0,ISO 3166-2:AX,Europe,Northern Europe,Northern Europe,150.0,154.0,NaN
2,Albania,AL,ALB,8.0,ISO 3166-2:AL,Europe,Southern Europe,Southern Europe,150.0,39.0,NaN
3,Algeria,DZ,DZA,12.0,ISO 3166-2:DZ,Africa,Northern Africa,Northern Africa,2.0,15.0,NaN
4,American Samoa,AS,ASM,16.0,ISO 3166-2:AS,Oceania,Polynesia,Polynesia,9.0,61.0,NaN


In [22]:
regions_fn = DATA_PATH+'raw/countrylist.csv'
regions = pd.read_csv(regions_fn)

cols = ['name','region','sub-region','intermediate-region']
regions = regions[cols]
regions.rename(columns={'name': 'CTR_MN_NM'}, inplace = True)
regions

,CTR_MN_NM,region,sub-region,intermediate-region
0,Afghanistan,Asia,Southern Asia,Southern Asia
1,Åland Islands,Europe,Northern Europe,Northern Europe
2,Albania,Europe,Southern Europe,Southern Europe
3,Algeria,Africa,Northern Africa,Northern Africa
4,American Samoa,Oceania,Polynesia,Polynesia
...,...,...,...,...
246,Yemen,Asia,Western Asia,Western Asia
247,Zambia,Africa,Sub-Saharan Africa,Eastern Africa
248,Zimbabwe,Africa,Sub-Saharan Africa,Eastern Africa
249,Swaziland,Africa,Sub-Saharan Africa,Southern Africa


### data check

In [ ]:
# Explore files - 1983
content = os.listdir(DATA_PATH+'interim/')
fns = sorted(glob(DATA_PATH+'interim/ERA5_HI/*csv'))
fn = fns[0]
df1983 = pd.read_csv(fn)
df1983.head()

In [ ]:
df1983.head()

### Let's check out the functions

In [ ]:
#### Step 1 - Function Loads all Tmax Data as an X-array
def read_data(dir_path, space_dim, time_dim):
    """ Function reads in all Tmax .csv files, joins them by date along the x-axis
    and returns the whole record as a x-array data array
    
    Args:   
        dir_path = path to .csv files 
        time_dim = name for time dim as a str ... use date :-)
        space_dim = col name for GHS-UCDB IDs as an str (ID_HDC_G0)
    """
    fn_list = sorted(glob.glob(dir_path+'*.csv'))
    df_out = pd.DataFrame()
    date_list = []

    # Open all Tmax files and concat into a df
    for i, fn in enumerate(fn_list):    
        # Open the CSV
        df = pd.read_csv(fn)

        # Get the city ids 
        if i == 1:
            df_id = df[space_dim]

        # get only the Tmax columns and concate date list 
        df_temp = df.iloc[:,3:] # get only temp columns
        date_list = date_list+list(df_temp.columns)

        # Drop cities w/ no temp record 
        df_temp_drop = df_temp.dropna()

        # Merge
        df_out = pd.concat([df_out, df_temp_drop], axis=1)
        print(df_out.shape)
    
    # make date into an array
    tmax_arr = df_out.to_numpy()

    # Make data into an xr.DataArray
    tmax_xr_da = xr.DataArray(tmax_arr, coords=[df_id, date_list], 
                             dims=[space_dim, time_dim])
    return tmax_xr_da

In [ ]:
def tmax_days(xarray, Tthresh):
    """ Function finds all the tmax days in a year and sums total days per year 
    greater than a threshold within a year where Tmax > Tthresh for each city. Returns the total number of days,
    the dates, the tempatures, and the intensity (daily Tmax - Tthresh)
    
    Args: 
        xarray = an xarray object with dims = (space, times)
        Tthresh = int of temp threshold
    """
    
    # empty lists & df
    id_list = []
    date_list = []
    tmax_list = []
    intensity_list = []
    df_out = pd.DataFrame()
    
    # subset xarray
    out = xarray.where(xarray > Tthresh, drop = True)

    # start loop 
    for index, loc in enumerate(out.ID_HDC_G0):
        id_list.append(out.ID_HDC_G0.values[index]) # get IDS
        date_list.append(out.sel(ID_HDC_G0 = loc).dropna(dim = 'date').date.values) # get event dates
        
        # #CPT 2020.02.23 
        # dayTot_list.append(len(out.sel(ID_HDC_G0 = loc).dropna(dim = 'date').date.values)) # get event totals
        
        tmax_list.append(out.sel(ID_HDC_G0 = loc).dropna(dim = 'date').values) # get temp values
        intensity_list.append(out.sel(ID_HDC_G0 = loc).dropna(dim = 'date').values - Tthresh) # get severity

    # write to a data frame
    df_out['ID_HDC_G0'] = id_list
    # df_out['total_days'] = dayTot_list #CPT 2020.02.23
    df_out['dates'] = date_list
    df_out['tmax'] = tmax_list
    df_out['tmax_tntensity'] = intensity_list

    # return df_out
    return df_out

In [ ]:
def event_split(dates, ID_HDC_G0, intensity, tmax): #, total_days): #CPT 2020.02.23
    
    """ Searchs a list of dates and isolates sequential dates as a list, then calculates event stats.
    See comments in code for more details. 
    
    Args:
        dates: pandas.core.index as julian dates
        ID_HDC_G0: city ID as string
        intensity: numpy.ndarray of intensities values
        tmax: numpy.ndarray of intensities values of tmax values
        total_days: total number of tmax days in a year for a given city

    """

    # city id
    city_id = ID_HDC_G0
    # tot_days = total_days #CPT 2020.02.23
    
    # lists to fill
    city_id_list = []
    # tot_days_list = [] #CPT 2020.02.23
    event_dates_list = []
    dur_list = []
    intensity_list = []
    tmax_list = []
    avg_temp_list = []
    avg_int_list = []
    tot_int_list = []
    year_list = []
    
    # data frame out
    df_out = pd.DataFrame()
    
    # turn days into julian days
    jul_days = jul_convert(dates)
    
    # Counters to make sure we write the correct event dates to a list, don't want julian days in output
    counter = 0
    start = 0
    end = 0
    
    # Loop through dur list and isolate seq days, temps, and intensities
    for k, g in groupby(enumerate(jul_days.values), lambda x: x[1]-x[0]):
        
        seq = list(map(itemgetter(1), g)) # isolate seq. days
        dur = len(seq) # duration of each event
        
        counter = counter + dur # add duration to counter
        end = counter # end of current event
        
        event_dates = dates[start:end] # dates of tmax days during each event
        intense = intensity[start:end] # intensity of each day during event
        temp = tmax[start:end] # temp of each day during event
        avg_temp = mean(temp) # avg. temp during event
        avg_int = mean(intense) # avg. intensity during event
        tot_int = intense.sum() # total intensity during event
        
        start = counter # reset start to current end (e.g. counter)
        year = event_dates[0].split('.')[0]
        
        # fill lists
        city_id_list.append(city_id)
        year_list.append(year)
        # tot_days_list.append(tot_days) #CPT 2020.02.23
        dur_list.append(dur)
        event_dates_list.append(event_dates)
        intensity_list.append(intense)
        tmax_list.append(temp)
        avg_temp_list.append(avg_temp)
        avg_int_list.append(avg_int)
        tot_int_list.append(tot_int)

    # write out as a dateframe
    df_out['ID_HDC_G0'] = city_id_list
    df_out['year'] = year_list
    # df_out['total_days'] = tot_days_list #CPT 2020.02.23
    df_out['duration'] = dur_list
    df_out['avg_temp'] = avg_temp_list
    df_out['avg_intensity'] = avg_int_list
    df_out['tot_intensity'] = tot_int_list
    df_out['event_dates'] = event_dates_list
    df_out['duration'] = dur_list
    df_out['intensity'] = intensity_list
    df_out['tmax'] = tmax_list

    return df_out

In [ ]:
#### Step 4 function feeds output from function 3 into function 4
def tmax_stats(df_in):
    """ runs event_split functionon a dataframe to produce desired tmax stats

        NOTE - If you add arguments to event_split to make more states,
        be sure to update this function

        args:
            df: input dataframe
    """
    df_out = pd.DataFrame()

    # NOTE - If you add arguments to event_split to make more stats,
    # be sure to update this function

    for index, row in df_in.iterrows():
        dates = row['dates'] # Get event dates
        intensity = row['tmax_tntensity'] # Get intensity for each day
        tmax = row['tmax'] # Get tmax for each day
        ID_HDC_G0 = row['ID_HDC_G0'] # get city id
        # total_days = row['total_days'] # get total number of tmax days -- CPT 2020.02.23

        df = event_split(dates, ID_HDC_G0, intensity, tmax)# , total_days) #CPT 2020.02.23

        df_out = df_out.append(df)

    return df_out

In [ ]:
def jul_convert(dates):
    "Function turn days into julian datetime"
    jul_days = pd.to_datetime(dates).to_julian_date()
    
    return jul_days

## Ok let's see how this works

In [ ]:
# Make data into an xr.DataArray
df_id = list(df1983['ID_HDC_G0'])
date_list = list(df1983.columns[3:])
df_temp = df1983.iloc[:,3:] # get only temp columns
df_temp_drop = df_temp.dropna() # drop cities with no temp record
space_dim = 'ID_HDC_G0'
time_dim = 'date'

tmax_arr = df_temp_drop.to_numpy()

tmax_xr_da = xr.DataArray(tmax_arr, coords=[df_id, date_list], 
                     dims=[space_dim, time_dim])

In [ ]:
df1983_406 = tmax_days(tmax_xr_da, 40.6)
df1983_406.head(1)

In [ ]:
df1983_406_STATS = tmax_stats(df1983_406)

In [ ]:
df1983_406_STATS.dtypes

In [ ]:
for item in df1983_406_STATS.iloc[0,:].values:
    print(type(item))

## try json for 2 day

In [ ]:
fn_nm = DATA_PATH+'interim/jsontest.json'
df1983_406_STATS.to_json(fn_nm, orient = 'split', index = False)

In [ ]:
import json
with open(fn_nm) as json_data:
    data = json.load(json_data)

In [ ]:
df = pd.read_json(fn_nm, orient='split')

In [ ]:
type(df['event_dates'][0])

In [ ]:
def find_two(df, fn_out):
    
    "find two day events buddy"
    print(len(df))
    df_out = df[df['duration'] > 1]
    print(len(df_out))
    #df_out.to_json(fn_out, orient = 'split', index = False)

In [ ]:
fn_out = DATA_PATH+'interim/jsontest2day.json'
find_two(df, fn_out)

In [ ]:
# as a loop
fns = sorted(glob(DATA_PATH+'interim/ERA5_STATS/*csv'))


for fn in fns:
    
    fn_out = fn.split('STATS/')[1].split('.csv')[0]
    fn_out = fn_out+'-2d406.json'
    print(fn_out)
    
    df = pd.read_csv(fn)
    print(len(df))
    df_out = df[df['duration'] > 1]
    print(len(df_out))
#     df_out = find_two(fn, fn_out)
#     print('\n')

## Ok now find 115

In [ ]:
df = pd.read_csv(fns[0])

In [ ]:
def tmax_float(tmax):
    tmaxfloat = float(tmax.split('[')[1].split(']')[0])
    return tmaxfloat

In [ ]:
def test(value):
    out = value
    out = float(out.split('[')[1].split(']')[0])
    return out

In [ ]:
# facts['pop2050'] = facts.apply(lambda row: final_pop(row['population'],row['population_growth']),axis=1)
df.apply(lambda row: test(row['tmax']), axis = 1)

In [ ]:
import pandas as pd
import sqlite3
conn = sqlite3.connect('factbook.db')
query = "select * from facts where area_land =0;"
facts = pd.read_sql_query(query,conn)
print(list(facts.columns.values))

def final_pop(initial_pop,growth_rate):
    final = initial_pop*math.e**(growth_rate*35)
    return(final)

facts['pop2050'] = facts['population','population_growth'].apply(final_pop,axis=1)
facts['pop2050'] = facts.apply(lambda row: final_pop(row['population'],row['population_growth']),axis=1)
